In [1]:
cd ..

In [2]:
%load_ext autoreload
%autoreload
%load_ext dotenv
%dotenv
import pandas as pd
from src.loaders.load_data import load_data

In [3]:
train, test, data_config = load_data('codrna')
train_df = pd.DataFrame(train).sample(frac=0.05, random_state=42)
X = train_df.iloc[:, :-1].values
print(X.shape)
train_df

In [137]:
from sklearn.datasets import load_breast_cancer
housing = load_breast_cancer()
frac = 1
X = pd.DataFrame(housing.data)
y = pd.DataFrame(housing.target)
df = pd.concat([X, y], axis=1)
df = df.sample(frac=frac, random_state=42)
X = df.iloc[:, :-1].values
X = (X - X.min(axis = 1, keepdims=True)) / (X.max(axis = 1, keepdims=True) - X.min(axis = 1, keepdims=True) + 0.001)
y = df.iloc[:, -1].values
print(X.shape)

In [138]:
df

In [151]:
datas = []
for i in range(10):
    datas.append(df.iloc[i, :].values)

for item in zip(*datas):
    string = "{:10.2f} "*len(item)
    print(string.format(*item))

In [139]:
from sklearn.decomposition import TruncatedSVD
embedding = TruncatedSVD(n_components=2)
X_transformed = embedding.fit_transform(X)
print(X_transformed.shape)

In [135]:
from sklearn.manifold import TSNE
embedding = TSNE(n_components=2, perplexity=50, random_state=42)
X_transformed = embedding.fit_transform(X)
print(X_transformed.shape)

In [133]:
from sklearn.manifold import SpectralEmbedding
embedding = SpectralEmbedding(n_components=2, random_state=42)
X_transformed = embedding.fit_transform(X)
print(X_transformed.shape)

In [140]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(x=X_transformed[:, 0], y=X_transformed[:, 1], hue = y, palette='viridis')
plt.show()

In [163]:
import numpy as np
corr = pd.DataFrame(X).corr().abs()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, annot=False, cmap='viridis')
plt.show()

In [197]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
# embedding = TruncatedSVD(n_components=2)
# X_transformed = embedding.fit_transform(X)
# print(X_transformed.shape)

pca = PCA(n_components=5, random_state=42)
X_transformed = pca.fit_transform(X)
print(X_transformed.shape)

corr_with_component = np.zeros((X_transformed.shape[1], X.shape[1]))
for components_id in range(X_transformed.shape[1]):
    corr_with_component[components_id, : ] = pd.DataFrame(X).corrwith(pd.Series(X_transformed[:, components_id])).values
fig, ax = plt.subplots()   
sns.heatmap(corr_with_component, annot=False, ax=ax)

plt.show()

In [201]:
na_inds = np.where(X_transformed[:, 4] > X_transformed[:, 4].mean())
na_inds

## MIWAE

In [178]:
from sklearn.datasets import make_gaussian_quantiles, make_hastie_10_2, make_blobs, make_classification
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X,y = make_classification(n_samples = 1000,  n_features=6, n_informative=2, n_redundant=1, random_state = 42)
X = StandardScaler().fit_transform(X)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
print(X.shape, y.shape)

In [179]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

pca = PCA(n_components=2, random_state=42)
X_transformed = pca.fit_transform(X)

import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(x=X_transformed[:, 0], y=X_transformed[:, 1], hue = y, palette='viridis')
plt.show()

In [180]:
sns.heatmap(pd.DataFrame(X).corr())

In [488]:
# generate missing data
import numpy as np

missing_ratio = 0.5
X1 = X.copy()
X_missing1 = X.copy()
np.random.seed(42)
X_missing1[X_missing1[:, 0] < np.quantile(X_missing1[:, 0], 0.5), 0] = np.nan
X_missing1[X_missing1[:, 1] > np.quantile(X_missing1[:, 1], 0.5), 1] = np.nan
X_missing1[X_missing1[:, 2] < np.quantile(X_missing1[:, 2], 0.5), 2] = np.nan
X_missing1[X_missing1[:, 3] > np.quantile(X_missing1[:, 3], 0.5), 3] = np.nan
X_missing1[X_missing1[:, 4] < np.quantile(X_missing1[:, 4], 0.5), 4] = np.nan
X_missing1[X_missing1[:, 5] > np.quantile(X_missing1[:, 5], 0.5), 5] = np.nan
mask1 = np.isnan(X_missing1)

# missing pattern
pattern = pd.DataFrame(mask1).astype(int).astype(str).apply(lambda x: ''.join(x), axis=1)
print(pattern.value_counts())

sorted_mask1 = pd.DataFrame(mask1).reindex(pattern.sort_values().index).values
mask1 = sorted_mask1
X1 = pd.DataFrame(X1).reindex(pattern.sort_values().index).values
X_missing1 = pd.DataFrame(X_missing1).reindex(pattern.sort_values().index).values
X_imp1 = X_missing1.copy()
X_imp1[mask1] = 0
sns.heatmap(mask1)

In [489]:
# generate missing data
import numpy as np

X2 = X.copy()
X_missing2 = X.copy()
np.random.seed(42)
X_missing2[X_missing2[:, 0] > np.quantile(X_missing2[:, 0], 0.5), 0] = np.nan
X_missing2[X_missing2[:, 1] < np.quantile(X_missing2[:, 1], 0.5), 1] = np.nan
X_missing2[X_missing2[:, 2] > np.quantile(X_missing2[:, 2], 0.5), 2] = np.nan
X_missing2[X_missing2[:, 3] < np.quantile(X_missing2[:, 3], 0.5), 3] = np.nan
X_missing2[X_missing2[:, 4] > np.quantile(X_missing2[:, 4], 0.5), 4] = np.nan
X_missing2[X_missing2[:, 5] < np.quantile(X_missing2[:, 5], 0.5), 5] = np.nan
mask2 = np.isnan(X_missing2)

# missing pattern
pattern = pd.DataFrame(mask2).astype(int).astype(str).apply(lambda x: ''.join(x), axis=1)
print(pattern.value_counts())

sorted_mask2 = pd.DataFrame(mask2).reindex(pattern.sort_values().index).values
mask2 = sorted_mask2
X2 = pd.DataFrame(X2).reindex(pattern.sort_values().index).values
X_missing2 = pd.DataFrame(X_missing2).reindex(pattern.sort_values().index).values
X_imp2 = X_missing2.copy()
X_imp2[mask2] = 0
sns.heatmap(mask2)

In [490]:
# stdlib
from typing import Any, List, Dict, Tuple

# third party
import numpy as np
import torch
from torch import nn, optim
import torch.distributions as td

# hyperimpute absolute
from emf.reproduce_utils import set_seed
from src.imputation.models.vae_models.decoder import GaussianDecoder, BernoulliDecoder, StudentTDecoder
from src.imputation.models.vae_models.encoder import BaseEncoder

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def weights_init(layer: Any) -> None:
    if type(layer) == nn.Linear:
        torch.nn.init.orthogonal_(layer.weight)


class MIWAE(nn.Module):
    """MIWAE imputation plugin

    Args:
        n_epochs: int
            Number of training iterations
        batch_size: int
            Batch size
        latent_size: int
            dimension of the latent space
        n_hidden: int
            number of hidden units
        K: int
            number of IS during training
        random_state: int
            random seed

    Reference: "MIWAE: Deep Generative Modelling and Imputation of Incomplete Data", Pierre-Alexandre Mattei,
    Jes Frellsen
    Original code: https://github.com/pamattei/miwae
    """

    def __init__(
            self,
            num_features: int,
            latent_size: int = 1,
            n_hidden: int = 16,
            n_hidden_layers: int = 2,
            seed: int = 0,
            out_dist='studentt',
            K: int = 20,
            L: int = 1000,
    ) -> None:

        super().__init__()
        set_seed(seed)

        # parameters
        self.num_features = num_features
        self.n_hidden = n_hidden  # number of hidden units in (same for all MLPs)
        self.n_hidden_layers = n_hidden_layers  # number of hidden layers in (same for all MLPs)
        self.latent_size = latent_size  # dimension of the latent space
        self.K = K  # number of IS during training
        self.L = L  # number of samples for imputation

        # encoder
        self.encoder = BaseEncoder(
            self.num_features, self.latent_size, [self.n_hidden for _ in range(self.n_hidden_layers)],
        ).to(DEVICE)

        # decoder
        self.out_dist = out_dist
        if out_dist == 'studentt':
            self.decoder = StudentTDecoder(
                self.latent_size, self.num_features, [self.n_hidden for _ in range(self.n_hidden_layers)],
            )
        elif out_dist == 'gaussian':
            self.decoder = GaussianDecoder(
                self.latent_size, self.num_features, [self.n_hidden for _ in range(self.n_hidden_layers)],
            )
        elif out_dist == 'bernoulli':
            self.decoder = BernoulliDecoder(
                self.latent_size, self.num_features, [self.n_hidden for _ in range(self.n_hidden_layers)],
            )
        else:
            raise ValueError("Invalid output distribution")

        self.decoder = self.decoder.to(DEVICE)
        
        # mapping z to mask using hint
        self.mask_net = nn.Sequential(
            nn.Linear(self.latent_size, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.num_features),
            nn.Sigmoid()
        ).to(DEVICE)
        
        self.z_L = None
        self.mask_L = None

        # prior for z
        self.p_z = td.Independent(
            td.Normal(loc=torch.zeros(self.latent_size).to(DEVICE), scale=torch.ones(self.latent_size).to(DEVICE)), 1
        )

    @staticmethod
    def name() -> str:
        return "miwae"

    def init(self, seed):
        set_seed(seed)
        self.encoder.apply(weights_init)
        self.decoder.apply(weights_init)
        self.mask_net.apply(weights_init)

    def compute_loss(self, inputs: List[torch.Tensor]) -> Tuple[torch.Tensor, Dict]:
        x, mask = inputs  # x - data, mask - missing mask
        batch_size = x.shape[0]

        # encoder
        mu, logvar = self.encoder(x)

        q_zgivenxobs = td.Independent(td.Normal(loc=mu, scale=logvar), 1)
        zgivenx = q_zgivenxobs.rsample([self.K])  # shape (K, batch_size, latent_size)
        zgivenx_flat = zgivenx.reshape([self.K * batch_size, self.latent_size])
        
        self.z_L = zgivenx_flat

        # decoder
        out_decoder = self.decoder(zgivenx_flat)
        recon_x_means = self.decoder.l_out_mu(out_decoder)

        # compute loss
        data_flat = torch.Tensor.repeat(x, [self.K, 1]).reshape([-1, 1]).to(DEVICE)
        tiled_mask = torch.Tensor.repeat(mask, [self.K, 1]).to(DEVICE)
        self.mask_L = tiled_mask

        # p(x|z)
        all_log_pxgivenz_flat = self.decoder.dist_xgivenz(out_decoder, flat=True).log_prob(data_flat)
        all_log_pxgivenz = all_log_pxgivenz_flat.reshape([self.K * batch_size, self.num_features])
        logpxobsgivenz = torch.sum(all_log_pxgivenz * tiled_mask, 1).reshape([self.K, batch_size])

        # p(z) and q(z|x)
        logpz = self.p_z.log_prob(zgivenx)
        logq = q_zgivenxobs.log_prob(zgivenx)

        neg_bound = -torch.mean(torch.logsumexp(logpxobsgivenz + logpz - logq, 0))

        return neg_bound, {}
    
    def mask_prediction_loss(self, x, mask):
        
        with torch.no_grad():
            batch_size = x.shape[0]
            mu, logvar = self.encoder(x)
            q_zgivenxobs = td.Independent(td.Normal(loc=mu, scale=logvar), 1)
            zgivenx = q_zgivenxobs.rsample([self.K])  # shape (K, batch_size, latent_size)
            zgivenx_flat = zgivenx.reshape([self.K * batch_size, self.latent_size])
            z = zgivenx_flat
            tiled_mask = torch.Tensor.repeat(mask, [self.K, 1]).to(DEVICE)
            mask = tiled_mask.reshape([-1, self.num_features]).int()
            mask = mask.float()
            #z = torch.cat([z, mask], 1)
        
        mask_pred = self.mask_net(z)
        loss = torch.nn.BCELoss()(mask_pred, mask)  # predict missing prob -> 1
        
        return loss
    
    def mask_prediction(self, x: torch.Tensor, mask: torch.Tensor, K: int = 10):
        
        with torch.no_grad():
            mu, logvar = self.encoder(x)
            q_zgivenxobs = td.Independent(td.Normal(loc=mu, scale=logvar), 1)
            zgivenx = q_zgivenxobs.rsample([K])  # shape (K, batch_size, latent_size)
            zgivenx_flat = zgivenx.reshape([K * x.shape[0], self.latent_size])
            mask_L = torch.Tensor.repeat(mask, [K, 1])
            #zgivenx_flat = torch.cat([zgivenx_flat, mask_L.reshape([-1, self.num_features]).float()], 1)
            
            mask_pred = self.mask_net(zgivenx_flat)
            mask_pred = mask_pred.reshape([K, x.shape[0], self.num_features])
            
            return mask_pred

    def impute(self, x: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        L = self.L
        batch_size = x.shape[0]
        p = x.shape[1]

        # encoder
        self.encoder.to(DEVICE)
        self.decoder.to(DEVICE)
        mu, logvar = self.encoder(x)
        q_zgivenxobs = td.Independent(td.Normal(loc=mu, scale=logvar), 1)

        zgivenx = q_zgivenxobs.rsample([L])
        zgivenx_flat = zgivenx.reshape([L * batch_size, self.latent_size])

        # decoder
        out_decoder = self.decoder(zgivenx_flat)
        recon_x_means = self.decoder.l_out_mu(out_decoder)

        # loss
        data_flat = torch.Tensor.repeat(x, [L, 1]).reshape([-1, 1]).to(DEVICE)
        tiledmask = torch.Tensor.repeat(mask, [L, 1]).to(DEVICE)

        all_log_pxgivenz_flat = self.decoder.dist_xgivenz(out_decoder, flat=True).log_prob(data_flat)
        all_log_pxgivenz = all_log_pxgivenz_flat.reshape([L * batch_size, p])

        logpxobsgivenz = torch.sum(all_log_pxgivenz * tiledmask, 1).reshape([L, batch_size])
        logpz = self.p_z.log_prob(zgivenx)
        logq = q_zgivenxobs.log_prob(zgivenx)

        # imputation weighted samples
        imp_weights = torch.nn.functional.softmax(
            logpxobsgivenz + logpz - logq, 0
        )  # these are w_1,....,w_L for all observations in the batch

        xgivenz = self.decoder.imp_dist_xgivenz(out_decoder)
        xms = xgivenz.sample().reshape([L, batch_size, p])
        xm = torch.einsum("ki,kij->ij", imp_weights, xms)

        # merge imputed values with observed values
        xhat = torch.clone(x)
        xhat[~mask.bool()] = xm[~mask.bool()]

        return xhat


In [491]:
from copy import deepcopy
from collections import OrderedDict
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def train(model, X, mask, train_params, X_true, return_params = False):
    
    model.to(DEVICE)
    lr = train_params['lr']
    weight_decay = train_params['weight_decay']
    epochs = train_params['epochs']
    batch_size = train_params['batch_size']
    verbose = train_params['verbose']
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay, amsgrad=True)

    # data
    n = X.shape[0]
    X_imp = X.copy()
    X_mask = mask.copy()
    bs = min(batch_size, n)

    final_loss = 0
    rmses = []
    for ep in range(epochs):
        
        # evaluation
        with torch.no_grad():
            X_imp_new = model.impute(
                torch.from_numpy(X_imp).float().to(DEVICE), torch.from_numpy(~X_mask).float().to(DEVICE)
            )
            rmse_value = rmse(X_imp_new.detach().clone().cpu().numpy(), X_true, X_mask)
            rmses.append(rmse_value)

        # shuffle data
        perm = np.random.permutation(n)  # We use the "random reshuffling" version of SGD
        batches_data = np.array_split(X_imp[perm,], int(n / bs), )
        batches_mask = np.array_split(X_mask[perm,], int(n / bs), )
        total_loss, total_iters = 0, 0
        total_mask_loss = 0
        model.train()
        for it in range(len(batches_data)):
            optimizer.zero_grad()
            model.encoder.zero_grad()
            model.decoder.zero_grad()
            model.mask_net.zero_grad()
            b_data = torch.from_numpy(batches_data[it]).float().to(DEVICE)
            b_mask = torch.from_numpy(~batches_mask[it]).float().to(DEVICE)
            data = [b_data, b_mask]
            loss, ret_dict = model.compute_loss(data)

            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            total_iters += 1


        # print loss
        if (ep + 1) % verbose == 0:
            print('Epoch %s/%s, Loss = %s RMSE = %s' % (ep, epochs, total_loss / total_iters, rmses[-1]))

        if DEVICE == "cuda":
            torch.cuda.empty_cache()
        
        final_loss = total_loss / total_iters

    model.to("cpu")
    
    if return_params:
        return deepcopy(model.state_dict())
    else:
        return model

def impute(model, X, mask):
    model.to(DEVICE)
    X_imp = X.copy()
    X_mask = mask.copy()
    X_imp[mask] = 0
    X_imp = torch.from_numpy(X_imp).float().to(DEVICE)
    X_mask = torch.from_numpy(~X_mask).float().to(DEVICE)
    with torch.no_grad():
        X_imp_new = model.impute(X_imp, X_mask)
    model.to("cpu")
    return X_imp_new.detach().clone().cpu().numpy()

In [492]:
def train_mask_net(model, X, mask, train_params):
    
    model.to(DEVICE)
    lr = train_params['lr']
    weight_decay = train_params['weight_decay']
    epochs = train_params['epochs']
    batch_size = train_params['batch_size']
    verbose = train_params['verbose']
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay, amsgrad=True)

    # data
    n = X.shape[0]
    X_imp = X.copy()
    X_mask = mask.copy()
    bs = min(batch_size, n)

    final_loss = 0
    rmses = []
    for ep in range(epochs):

        # shuffle data
        perm = np.random.permutation(n)  # We use the "random reshuffling" version of SGD
        batches_data = np.array_split(X_imp[perm,], int(n / bs), )
        batches_mask = np.array_split(X_mask[perm,], int(n / bs), )
        total_loss, total_iters = 0, 0
        total_mask_loss = 0
        model.train()
        for it in range(len(batches_data)):
            optimizer.zero_grad()
            model.mask_net.zero_grad()
            b_data = torch.from_numpy(batches_data[it]).float().to(DEVICE)
            b_mask = torch.from_numpy(batches_mask[it]).float().to(DEVICE)
            loss = model.mask_prediction_loss(b_data, b_mask)

            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            total_iters += 1


        # print loss
        if (ep + 1) % verbose == 0:
            print('Epoch %s/%s, Mask Loss = %s' % (ep, epochs, total_loss / total_iters))

        if DEVICE == "cuda":
            torch.cuda.empty_cache()
        
        final_loss = total_loss / total_iters

    model.to("cpu")
    
    return model

In [443]:
imp_model1 = MIWAE(
    num_features=X1.shape[1], latent_size = 3, n_hidden = 16, n_hidden_layers = 2, out_dist = 'gaussian', K = 20, L = 100
)
train_params = {
    'lr': 1e-3,
    'weight_decay': 1e-5,
    'epochs': 500,
    'batch_size': 64,
    'verbose': 10
}

imp_model1 = train(imp_model1, X_imp1, mask1, train_params, X)
X_imp_new1 = impute(imp_model1, X_imp1, mask1)
print(X_imp_new1.shape)
from src.evaluation.imp_quality_metrics import rmse
print(rmse(X_imp_new1, X1, mask1))

In [246]:
imp_model2 = MIWAE(
    num_features=X2.shape[1], latent_size = 3, n_hidden = 16, n_hidden_layers = 2, out_dist = 'gaussian', K = 20, L = 100
)
train_params = {
    'lr': 1e-3,
    'weight_decay': 1e-5,
    'epochs': 500,
    'batch_size': 64,
    'verbose': 10
}

imp_model2 = train(imp_model2, X_imp2, mask2, train_params, X2)

X_imp_new2 = impute(imp_model2, X_imp2, mask2)
print(X_imp_new2.shape)
from src.evaluation.imp_quality_metrics import rmse
print(rmse(X_imp_new2, X2, mask2))

In [247]:
X_imp_new1 = impute(imp_model2, X_imp1, mask1)
print(X_imp_new1.shape)
from src.evaluation.imp_quality_metrics import rmse
print(rmse(X_imp_new1, X1, mask1))

X_imp_new2 = impute(imp_model1, X_imp2, mask2)
print(X_imp_new2.shape)
from src.evaluation.imp_quality_metrics import rmse
print(rmse(X_imp_new2, X2, mask2))

In [493]:
def fedavg(local_model_parameters):
    # federated averaging implementation
    averaged_model_state_dict = OrderedDict()  # global parameters
    #sample_sizes = [item['sample_size'] for item in fit_res]

    for it, local_model_state_dict in enumerate(local_model_parameters):
        for key in local_model_state_dict.keys():
            if it == 0:
                averaged_model_state_dict[key] = (1/len(local_model_parameters))*local_model_state_dict[key]
            else:
                averaged_model_state_dict[key] += (1/len(local_model_parameters))*local_model_state_dict[key]

    # copy parameters for each client
    agg_model_parameters = [deepcopy(averaged_model_state_dict) for _ in range(len(local_model_parameters))]
    agg_res = {}

    return agg_model_parameters, agg_res

imp_model_global = MIWAE(
    num_features=X.shape[1], latent_size = 3, n_hidden = 16, n_hidden_layers = 2, out_dist = 'gaussian', K = 20, L = 100
)
imp_model1 = MIWAE(
    num_features=X1.shape[1], latent_size = 3, n_hidden = 16, n_hidden_layers = 2, out_dist = 'gaussian', K = 20, L = 100
)

imp_model2 = MIWAE(
    num_features=X2.shape[1], latent_size = 3, n_hidden = 16, n_hidden_layers = 2, out_dist = 'gaussian', K = 20, L = 100
)

imp_model1.load_state_dict(deepcopy(imp_model_global.state_dict()))
imp_model2.load_state_dict(deepcopy(imp_model_global.state_dict()))

global_iteration = 100

train_params = {
    'lr': 1e-3,
    'weight_decay': 1e-5,
    'epochs': 5,
    'batch_size': 64,
    'verbose': 10
}

for it in range(global_iteration):
    print(f"Global iteration {it}")
    # global model training
    params1 = train(imp_model_global, X_imp1, mask1, train_params, X1, return_params=True)
    params2 = train(imp_model_global, X_imp2, mask2, train_params, X2, return_params=True)
    
    # local model training
    imp_model1 = train(imp_model1, X_imp1, mask1, train_params, X1)
    imp_model2 = train(imp_model2, X_imp2, mask2, train_params, X2)
    
    # federated averaging
    avg_params, _ = fedavg([params1, params2])
    imp_model_global.load_state_dict(avg_params[0])
    
    # update local model encoder
    imp_model1.encoder.load_state_dict(imp_model_global.encoder.state_dict(), strict = False)
    imp_model2.encoder.load_state_dict(imp_model_global.encoder.state_dict(), strict = False)
    
    X_imp_g1 = impute(imp_model_global, X_imp1, mask1)
    X_imp_l1 = impute(imp_model1, X_imp1, mask1)
    X_imp_g2 = impute(imp_model_global, X_imp2, mask2)
    X_imp_l2 = impute(imp_model2, X_imp2, mask2)
    print(rmse(X_imp_g1, X1, mask1), rmse(X_imp_g2, X2, mask2))
    print(rmse(X_imp_l1, X1, mask1), rmse(X_imp_l2, X2, mask2))

In [497]:
train_params = {
    'lr': 1e-3,
    'weight_decay': 1e-5,
    'epochs': 60,
    'batch_size': 64,
    'verbose': 10
}

imp_model1 = train_mask_net(imp_model1, X_imp1, mask1, train_params)
imp_model2 = train_mask_net(imp_model2, X_imp2, mask2, train_params)

In [514]:
import torch.distributions as td

def new_impute(global_model, local_model, x: torch.Tensor, mask: torch.Tensor, L) -> torch.Tensor:
    
    with torch.no_grad():
        x = x.to(DEVICE)
        mask = ~mask.to(DEVICE)
        batch_size = x.shape[0]
        p = x.shape[1]
        
        # encoder
        global_model.encoder.to(DEVICE)
        global_model.decoder.to(DEVICE)
        local_model.encoder.to(DEVICE)
        local_model.decoder.to(DEVICE)
        local_model.mask_net.to(DEVICE)
        mu, logvar = local_model.encoder(x)
        
        q_zgivenxobs = td.Independent(td.Normal(loc=mu, scale=logvar), 1)
        zgivenx = q_zgivenxobs.rsample([L])
        zgivenx_flat = zgivenx.reshape([L * batch_size, local_model.latent_size])
    
        # decoder
        global_out_decoder = global_model.decoder(zgivenx_flat)
        local_out_decoder = local_model.decoder(zgivenx_flat)
        #recon_x_means = self.decoder.l_out_mu(out_decoder)
    
        # loss
        data_flat = torch.Tensor.repeat(x, [L, 1]).reshape([-1, 1]).to(DEVICE)
        tiledmask = torch.Tensor.repeat(mask, [L, 1]).to(DEVICE)
    
        all_log_pxgivenz_flat_local = local_model.decoder.dist_xgivenz(local_out_decoder, flat=True).log_prob(data_flat)
        all_log_pxgivenz_local = all_log_pxgivenz_flat_local.reshape([L * batch_size, p])
        logpxobsgivenz = torch.sum(all_log_pxgivenz_local * tiledmask, 1).reshape([L, batch_size])
        
        all_log_pxgivenz_flat_global = global_model.decoder.dist_xgivenz(global_out_decoder, flat=True).log_prob(data_flat)
        all_log_pxgivenz_global = all_log_pxgivenz_flat_global.reshape([L * batch_size, p])
        logpxobsgivenz_global = torch.sum(all_log_pxgivenz_global * tiledmask, 1).reshape([L, batch_size])
        
        #print(all_log_pxgivenz_global.exp(), all_log_pxgivenz_local.exp())
        
        logpz = local_model.p_z.log_prob(zgivenx)
        logq = q_zgivenxobs.log_prob(zgivenx)
        
        logpz_global = global_model.p_z.log_prob(zgivenx)
        logq_global = q_zgivenxobs.log_prob(zgivenx)
            
        # imputation weighted samples
        imp_weights_local = torch.nn.functional.softmax(
            logpxobsgivenz + logpz - logq, 0
        )  # these are w_1,....,w_L for all observations in the batch
        imp_weights_global = torch.nn.functional.softmax(
            logpxobsgivenz_global + logpz_global - logq_global, 0
        )
    
        xgivenz_local = local_model.decoder.imp_dist_xgivenz(local_out_decoder)
        xgivenz_global = global_model.decoder.imp_dist_xgivenz(global_out_decoder)
        xms_local = xgivenz_local.sample().reshape([L, batch_size, p])
        xms_global = xgivenz_global.sample().reshape([L, batch_size, p])
        
        mask_prob = local_model.mask_net(zgivenx_flat)
        mask_prob = mask_prob.reshape([L, batch_size, p]) + 0.01  # avoid zero
        
        xm_local = torch.einsum("ki,kij->ij", imp_weights_local, (1 - 1/mask_prob)*xms_local)
        xm_global = torch.einsum("ki,kij->ij", imp_weights_local, 1/mask_prob*xms_global)
        
        xm = xm_global + xm_local
    
        # merge imputed values with observed values
        xhat = torch.clone(x)
        xhat[~mask.bool()] = xm[~mask.bool()]
    
        return xhat.detach().clone().cpu().numpy()

In [515]:
X_imp_new1 = new_impute(imp_model_global, imp_model1, torch.from_numpy(X_imp1).float(), torch.from_numpy(mask1), 1000)
print(rmse(X_imp_new1, X1, mask1))

X_imp_new2 = new_impute(imp_model_global, imp_model2, torch.from_numpy(X_imp2).float(), torch.from_numpy(mask2), 1000)
print(rmse(X_imp_new2, X2, mask2))

In [500]:
X_imp_new1 = new_impute(imp_model_global, imp_model1, torch.from_numpy(X_imp1[-4:,:]).float(), torch.from_numpy(mask1[-4:,:]), 5)
X_imp_g1 = impute(imp_model_global, X_imp1[-4:,:], mask1[-4:,:])
X_imp_l1 = impute(imp_model1, X_imp1[-4:,:], mask1[-4:,:])

In [501]:
X[-4:]

In [458]:
X_imp_new1

In [311]:
X_imp_g1

In [459]:
X_imp_l1